# Files for unit test

In [ ]:
import numpy as np
import healpy as hp
import astropy.units as u

from histpy import Histogram

from cosipy import test_data
from cosipy.image_deconvolution import CoordsysConversionMatrix, SpacecraftAttitudeExposureTable, AllSkyImageModel, DataIF_COSI_DC2
from cosipy.spacecraftfile import SpacecraftFile

from cosipy.response import FullDetectorResponse

In [ ]:
nside = 1

In [ ]:
ori = SpacecraftFile.parse_from_file(test_data.path / "20280301_first_10sec.ori")
ori

# Exposure Table

In [ ]:
nside = 1

exposure_table = SpacecraftAttitudeExposureTable.from_orientation(ori, nside=nside, start=ori.get_time()[0], stop=ori.get_time()[-1], min_exposure=0, min_num_pointings=1)

exposure_table

In [ ]:
exposure_table.to_pickle(test_data.path / "image_deconvolution" / "exposure_table_test_nside1_ring.pickle")

In [ ]:
exposure_table.save_as_fits(test_data.path / "image_deconvolution" / "exposure_table_test_nside1_ring.fits", overwrite=True)

In [ ]:
map_pointing_zx = exposure_table.calc_pointing_trajectory_map()

In [ ]:
map_pointing_zx.write(test_data.path / "image_deconvolution" / "map_pointing_zx_test_nside1_ring.hdf5", overwrite=True)

In [ ]:
hp.mollview(map_pointing_zx.contents[:,0].todense())

# CoordsysConversionMatrix using time bins

In [ ]:
full_detector_response = FullDetectorResponse.open(test_data.path / "test_full_detector_response.h5")

In [ ]:
full_detector_response

In [ ]:
ccm = CoordsysConversionMatrix.time_binning_ccm(full_detector_response, ori, [ori.get_time()[0].value, ori.get_time()[-1].value] * u.s)

In [ ]:
ccm.axes['Time'].edges

In [ ]:
ccm.write(test_data.path / "image_deconvolution" / 'ccm_time_test.hdf5', overwrite=True)

# CoordsysConversionMatrix using ScAtt bins

In [ ]:
ccm = CoordsysConversionMatrix.spacecraft_attitude_binning_ccm(full_detector_response, exposure_table)

In [ ]:
ccm_use_averaged_pointing = CoordsysConversionMatrix.spacecraft_attitude_binning_ccm(full_detector_response, exposure_table, use_averaged_pointing = True)

In [ ]:
ccm.write(test_data.path / "image_deconvolution" / 'ccm_scatt_use_averaged_pointing_False_test.hdf5', overwrite=True)

In [ ]:
ccm_use_averaged_pointing.write(test_data.path / "image_deconvolution" / 'ccm_scatt_use_averaged_pointing_True_test.hdf5', overwrite=True)

# AllSkyImageModel

In [ ]:
model = AllSkyImageModel(nside = nside, energy_edges = [100.0, 1000.0] * u.keV)
model[:] = 1.0 / u.s / u.sr / u.cm**2

In [ ]:
model.write(test_data.path / "image_deconvolution" / 'all_sky_image_model_test_nside1.hdf5', overwrite=True)

# Histogram in ScAtt Binning

In [ ]:
from cosipy import response
from cosipy import test_data
from cosipy import BinnedData
from histpy import Histogram, Axes, Axis, HealpixAxis
from scoords import Attitude, SpacecraftFrame
from astropy.coordinates import SkyCoord, ICRS, Galactic, FK4, FK5
import astropy.units as u
import numpy as np
import healpy as hp

In [ ]:
full_detector_response = response.FullDetectorResponse.open(test_data.path / "test_full_detector_response.h5")
full_detector_response

In [ ]:
analysis = BinnedData(test_data.path / "inputs_crab.yaml")
analysis

In [ ]:
analysis.energy_bins = full_detector_response.axes['Em'].edges.to(u.keV).value
analysis.nside = full_detector_response.axes['PsiChi'].nside

assert np.all(full_detector_response.axes['Phi'].widths == full_detector_response.axes['Phi'].widths[0]) == True

analysis.phi_pix_size = full_detector_response.axes['Phi'].widths[0].to(u.deg).value

analysis.time_bins = 10 #s

In [ ]:
def get_binned_data_scatt(unbinned_event, exposure_table, psichi_binning = 'local', sparse = False):
    exposure_dict = {row['healpix_index']: row['scatt_binning_index'] for _, row in exposure_table.iterrows()}
        
    # from BinnedData.py
 
    # Get energy bins:
    energy_bin_edges = np.array(unbinned_event.energy_bins)
    
    # Get phi bins:
    number_phi_bins = int(180./unbinned_event.phi_pix_size)
    phi_bin_edges = np.linspace(0,180,number_phi_bins+1)
    
    # Define psichi axis and data for binning:
    if psichi_binning == 'galactic':
        psichi_axis = HealpixAxis(nside = unbinned_event.nside, scheme = unbinned_event.scheme, coordsys = 'galactic', label='PsiChi')
        coords = SkyCoord(l=unbinned_event.cosi_dataset['Chi galactic']*u.deg, b=unbinned_event.cosi_dataset['Psi galactic']*u.deg, frame = 'galactic')
    if psichi_binning == 'local':
        psichi_axis = HealpixAxis(nside = unbinned_event.nside, scheme = unbinned_event.scheme, coordsys = SpacecraftFrame(), label='PsiChi')
        coords = SkyCoord(lon=unbinned_event.cosi_dataset['Chi local']*u.rad, lat=((np.pi/2.0) - unbinned_event.cosi_dataset['Psi local'])*u.rad, frame = SpacecraftFrame())

    # Define scatt axis and data for binning
    n_scatt_bins = len(exposure_table)
    scatt_axis = Axis(np.arange(n_scatt_bins + 1), label='ScAtt')
    
    is_nest = True if exposure_table.scheme == 'nested' else False
    
    nside_scatt = exposure_table.nside
    
#    zindex = hp.ang2pix(nside_scatt, unbinned_event.cosi_dataset['Zpointings (glon,glat)'].T[0] * 180 / np.pi, 
#                        unbinned_event.cosi_dataset['Zpointings (glon,glat)'].T[1] * 180 / np.pi, nest=is_nest, lonlat=True)
#    xindex = hp.ang2pix(nside_scatt, unbinned_event.cosi_dataset['Xpointings (glon,glat)'].T[0] * 180 / np.pi, 
#                        unbinned_event.cosi_dataset['Xpointings (glon,glat)'].T[1] * 180 / np.pi, nest=is_nest, lonlat=True)    
#    scatt_data = np.array( [ exposure_dict[(z, x)] + 0.5 if (z,x) in exposure_dict.keys() else -1 for z, x in zip(zindex, xindex)] ) # should this "0.5" be needed?

    # NOTE: test_data.path / "unbinned_data_MEGAlib_calc.hdf5" is written in a old format!!!
    zindex = hp.ang2pix(nside_scatt, unbinned_event.cosi_dataset['Zpointings'].T[0] * 180 / np.pi, 
                        unbinned_event.cosi_dataset['Zpointings'].T[1] * 180 / np.pi, nest=is_nest, lonlat=True)
    xindex = hp.ang2pix(nside_scatt, unbinned_event.cosi_dataset['Xpointings'].T[0] * 180 / np.pi, 
                        unbinned_event.cosi_dataset['Xpointings'].T[1] * 180 / np.pi, nest=is_nest, lonlat=True)    
    scatt_data = np.array( [ exposure_dict[(z, x)] + 0.5 if (z,x) in exposure_dict.keys() else -1 for z, x in zip(zindex, xindex)] ) # should this "0.5" be needed?
    
    # Initialize histogram:
    binned_data = Histogram([scatt_axis,
                              Axis(energy_bin_edges*u.keV, label='Em'),
                              Axis(phi_bin_edges*u.deg, label='Phi'),
                              psichi_axis],
                              sparse=sparse)

    # Fill histogram:
    binned_data.fill(scatt_data, unbinned_event.cosi_dataset['Energies']*u.keV, np.rad2deg(unbinned_event.cosi_dataset['Phi'])*u.deg, coords)    
    
    return binned_data

In [ ]:
analysis.cosi_dataset = analysis.get_dict_from_hdf5(test_data.path / "unbinned_data_MEGAlib_calc.hdf5")

In [ ]:
binned_signal = get_binned_data_scatt(analysis, exposure_table, psichi_binning = 'local', sparse = False)

In [ ]:
binned_signal.write(test_data.path / "image_deconvolution" / 'test_event_histogram_localCDS_scatt.h5', overwrite=True)